Importar Librerias y parametros iniciales

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import cv2
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Parámetros globales
IMAGE_SIZE = (32, 32)      # Tamaño al que se redimensionan las imágenes
CSV_PATH = "onion_dataset.csv"   # Ruta al CSV con image_path y class
FEATURES_OUTPUT = "onion_features.npz"  # Archivo donde se guardarán las features

Función para extraer características básicas

In [3]:
def extract_basic_features(image_path, image_size=IMAGE_SIZE):
    """
    Extrae características básicas de una imagen:
    - Píxeles aplanados y normalizados
    - Histogramas de color (R, G, B)
    - Estadísticas básicas por canal (media, std, min, max)
    - Características de textura usando gradientes de Sobel
    """
    try:
        # 1. Cargar imagen
        img = Image.open(image_path)

        # 2. Asegurar que está en RGB
        if img.mode != 'RGB':
            img = img.convert('RGB')

        # 3. Redimensionar
        img_resized = img.resize(image_size)
        img_array = np.array(img_resized)

        # 4. Píxeles aplanados (normalizados 0-1)
        pixels_flat = img_array.flatten() / 255.0

        # 5. Histogramas de color para cada canal (32 bins por canal)
        hist_features = []
        for channel in range(3):  # 0=R, 1=G, 2=B
            hist, _ = np.histogram(img_array[:, :, channel], bins=32, range=(0, 256))
            hist_features.extend(hist / hist.sum())  # normalizamos el histograma

        # 6. Estadísticas básicas por canal
        stats_features = []
        for channel in range(3):
            channel_data = img_array[:, :, channel]
            stats_features.extend([
                channel_data.mean() / 255.0,
                channel_data.std()  / 255.0,
                channel_data.min()  / 255.0,
                channel_data.max()  / 255.0
            ])

        # 7. Características de textura: gradientes en escala de grises
        gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)

        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

        texture_features = [
            np.mean(np.abs(sobelx)) / 255.0,
            np.std(np.abs(sobelx))  / 255.0,
            np.mean(np.abs(sobely)) / 255.0,
            np.std(np.abs(sobely))  / 255.0
        ]

        # 8. Combinar todas las características en un solo vector
        all_features = np.concatenate([
            pixels_flat,     # píxeles
            hist_features,   # histogramas
            stats_features,  # estadísticas por canal
            texture_features # textura
        ])

        return all_features

    except Exception as e:
        print(f"Error procesando {image_path}: {e}")
        return None

Función para extraer HOG

In [4]:
def extract_hog_features(image_path, image_size=IMAGE_SIZE):
    """
    Extrae características HOG (Histogram of Oriented Gradients).
    """
    try:
        from skimage.feature import hog

        # 1. Cargar usando OpenCV
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError("cv2.imread devolvió None (¿ruta incorrecta?)")

        # 2. Convertir de BGR a RGB y redimensionar
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img, image_size)

        # 3. Pasar a escala de grises
        gray = cv2.cvtColor(img_resized, cv2.COLOR_RGB2GRAY)

        # 4. Calcular HOG
        features = hog(
            gray,
            orientations=9,
            pixels_per_cell=(8, 8),
            cells_per_block=(2, 2),
            block_norm='L2-Hys',
            feature_vector=True
        )

        return features

    except Exception as e:
        print(f"Error extrayendo HOG de {image_path}: {e}")
        return None

Función para preparar el dataset

In [ ]:
def prepare_dataset_for_ml(
    csv_path,
    feature_type='basic',      # 'basic', 'hog' o 'combined'
    sample_size=None,
    image_size=IMAGE_SIZE
):
    """
    Lee el CSV, recorre todas las imágenes y construye:
      - X: matriz de características
      - y: vector de etiquetas codificadas
      - feature_names: nombres de columnas de X (solo detallados para 'basic')
      - label_encoder: para poder decodificar luego las clases
    """
    # 1. Leer CSV
    print("Leyendo CSV...")
    df = pd.read_csv(csv_path)

    # 2. Tomar una muestra del dataset
    if sample_size and sample_size < len(df):
        df = df.sample(n=sample_size, random_state=42)
        print(f"Usando muestra de {sample_size} imágenes de un total de {len(df)}")

    # 3. Inicializar listas
    features_list = []
    labels_list = []

    print(f"\nIniciando extracción de características ({feature_type})...")
    print("Esto puede tardar varios minutos según el tamaño del dataset.\n")

    # 4. Recorrer cada fila del CSV
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        image_path = row['image_path']
        class_label = row['class']

        # 4.1 Seleccionar tipo de característica
        if feature_type == 'basic':
            features = extract_basic_features(image_path, image_size=image_size)
        elif feature_type == 'hog':
            features = extract_hog_features(image_path, image_size=image_size)
        elif feature_type == 'combined':
            basic_feat = extract_basic_features(image_path, image_size=image_size)
            hog_feat   = extract_hog_features(image_path,  image_size=image_size)
            if basic_feat is not None and hog_feat is not None:
                features = np.concatenate([basic_feat, hog_feat])
            else:
                features = None
        else:
            raise ValueError(f"Tipo de característica no válido: {feature_type}")

        # 4.2 Almacenar si fue exitoso
        if features is not None:
            features_list.append(features)
            labels_list.append(class_label)

    # 5. Convertir a numpy arrays
    X = np.array(features_list)
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels_list)

    # 6. Info de salida
    print("\nCaracterísticas extraídas exitosamente:")
    print(f"   • Forma de X: {X.shape}")
    print(f"   • Número de clases: {len(label_encoder.classes_)}")
    print(f"   • Clases: {list(label_encoder.classes_)}")

    # 7. Crear nombres de características
    if feature_type == 'basic':
        n_pixels = image_size[0] * image_size[1] * 3
        feature_names = (
            [f'pixel_{i}' for i in range(n_pixels)] +
            [f'hist_r_{i}' for i in range(32)] +
            [f'hist_g_{i}' for i in range(32)] +
            [f'hist_b_{i}' for i in range(32)] +
            ['mean_r', 'std_r', 'min_r', 'max_r'] +
            ['mean_g', 'std_g', 'min_g', 'max_g'] +
            ['mean_b', 'std_b', 'min_b', 'max_b'] +
            ['gradient_x_mean', 'gradient_x_std', 'gradient_y_mean', 'gradient_y_std']
        )
    else:
        feature_names = [f'feature_{i}' for i in range(X.shape[1])]

    return X, y, feature_names, label_encoder

Función para guardar las características

In [6]:
def save_features(X, y, classes, output_path=FEATURES_OUTPUT):
    """
    Guarda X, y y las clases en un archivo .npz comprimido.
    """
    print(f"\nGuardando características en: {output_path}")
    np.savez_compressed(
        output_path,
        X=X,
        y=y,
        classes=classes
    )
    print("Archivo guardado correctamente.")


Flujo Principal

In [ ]:
# 1. Preparar dataset (extraer características y codificar etiquetas)
X, y, feature_names, label_encoder = prepare_dataset_for_ml(
    CSV_PATH,
    feature_type='basic',   # cambia a 'hog' o 'combined' si quieres
    image_size=IMAGE_SIZE
)

Leyendo CSV...

Iniciando extracción de características (basic)...
Esto puede tardar varios minutos según el tamaño del dataset.



100%|██████████| 13229/13229 [00:30<00:00, 430.96it/s]



Características extraídas exitosamente:
   • Forma de X: (13229, 3184)
   • Número de clases: 15
   • Clases: ['Alternaria_D', 'Botrytis Leaf Blight', 'Bulb Rot', 'Bulb_blight-D', 'Caterpillar-P', 'Downy mildew', 'Fusarium-D', 'Healthy leaves', 'Iris yellow virus_augment', 'Purple blotch', 'Rust', 'Virosis-D', 'Xanthomonas Leaf Blight', 'onion1', 'stemphylium Leaf Blight']


In [12]:
# 2. Guardar características en disco 
save_features(X, y, label_encoder.classes_, output_path=FEATURES_OUTPUT)


Guardando características en: onion_features.npz
Archivo guardado correctamente.


In [13]:
# 4. Dividir en entrenamiento y prueba
print("\nDividiendo en conjunto de entrenamiento y prueba...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


Dividiendo en conjunto de entrenamiento y prueba...


In [16]:
print("\nResumen de los conjuntos:")
print("-" * 80)
print("Conjunto de entrenamiento:")
print(f"  • X_train shape: {X_train.shape}")
print(f"  • y_train shape: {y_train.shape}")

print("\nConjunto de prueba:")
print(f"  • X_test shape: {X_test.shape}")
print(f"  • y_test shape: {y_test.shape}")

print("\nLas características extraídas incluyen:")
print("  - Valores de píxeles normalizados")
print("  - Histogramas de color (R, G, B)")
print("  - Estadísticas de color (media, std, min, max por canal)")
print("  - Características de textura (gradientes Sobel)")
print(f"Total de características por imagen: {X.shape[1]}")



Resumen de los conjuntos:
--------------------------------------------------------------------------------
Conjunto de entrenamiento:
  • X_train shape: (10583, 3184)
  • y_train shape: (10583,)

Conjunto de prueba:
  • X_test shape: (2646, 3184)
  • y_test shape: (2646,)

Las características extraídas incluyen:
  - Valores de píxeles normalizados
  - Histogramas de color (R, G, B)
  - Estadísticas de color (media, std, min, max por canal)
  - Características de textura (gradientes Sobel)
Total de características por imagen: 3184
